In [1]:
%cd /home/jaeheonshim/music-vibes

/home/jaeheonshim/music-vibes


In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Subset
import librosa
from vibenet.dataset import FMAVGGishDataset
from vibenet.models import VGGishMLP
from tqdm import tqdm
from torchvggish import vggish_input

In [3]:
labels = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']

In [4]:
dataset = FMAVGGishDataset('data/preprocessed/vggish')
print(dataset[0][0])
# dataset = Subset(dataset, list(range(50)))

N = len(dataset)
print(N)
train_size = int(N * 0.8)
train_ds, test_ds = random_split(dataset, [train_size, N - train_size], generator=torch.Generator().manual_seed(42))
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False)

[[[-2.7525146  -2.683021   -2.6432276  ... -3.9684596  -3.9523442
   -3.869654  ]
  [ 0.2586684   0.8037823   1.1073251  ... -2.2905943  -1.5651563
   -1.6188581 ]
  [ 0.42302436  1.05617     0.926886   ... -0.8017502  -1.0229014
   -1.076192  ]
  ...
  [ 0.88455707  0.4266815   1.2014482  ... -0.1787137  -0.5918389
    0.19124033]
  [ 1.2265248   1.0417923   0.40203607 ... -0.55309474 -0.64335
   -1.0096501 ]
  [ 1.2111201   1.3818171   2.2253103  ... -0.9067962  -1.1063846
   -0.70985943]]

 [[ 0.8623978   1.2762761   2.1428616  ... -0.6347559  -0.6723031
   -0.95313996]
  [ 1.1082301   1.233897    1.2746985  ... -0.9564062  -0.79365534
   -0.35816047]
  [ 1.0627131   1.342058    2.4580252  ... -0.79503554 -0.6214866
   -0.40497547]
  ...
  [ 2.5603573   2.0551615   1.9706583  ...  0.7395899  -0.07511103
   -0.7212197 ]
  [ 2.4735575   1.516179   -0.05098555 ...  0.9755231   0.33867064
    0.08468862]
  [ 0.9329136  -0.07456745  0.80708444 ...  1.183292    1.3899543
    1.3542749 ]]


In [ ]:
device = 'cuda'

model = VGGishMLP()
model = model.to(device)

NUM_EPOCHS = 100

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)

In [ ]:
best_loss = 1

for epoch in range(NUM_EPOCHS):
    scheduler.step()

    print(f"Epoch {epoch+1}:")
    model.train()
    
    train_losses = []

    with tqdm(train_dl, desc='Training') as pbar:
        for data, label in pbar:
            data, label = data.to(device).float(), label.to(device).float()
            optimizer.zero_grad()

            with torch.autocast(device_type="cuda"):
                pred = model(data)
                loss = F.l1_loss(pred, label)

            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            train_loss = np.mean(train_losses)

            pbar.set_postfix({'loss': f"{train_loss:.4f}", 'lr': f"{scheduler.get_last_lr()[0]}"})

    model.eval()

    eval_losses = []
    label_maes = []

    with tqdm(test_dl, desc='Validation') as pbar:
        with torch.inference_mode():
            for data, label in pbar:
                data, label = data.to(device).float(), label.to(device).float()

                pred = model(data)
                loss = F.l1_loss(pred, label)

                eval_losses.append(loss.item())
                eval_loss = np.mean(eval_losses)

                mae = torch.mean(torch.abs(pred - label), dim=0) # for per-metric MAE
                label_maes.append(mae.cpu().numpy())

                pbar.set_postfix({'loss': f"{eval_loss:.4f}"})
    
    eval_loss = np.mean(eval_losses)
    print("Validation loss:", eval_loss)

    label_mae = np.mean(label_maes, axis=0)
    print("MAE per label:")
    for i, label in enumerate(labels):
        print(f"\t{label:<20}{label_mae[i]:.4f}")

    if eval_loss < best_loss:
        best_loss = eval_loss
        torch.save(model.state_dict(), 'best_model.pt')
        print('Saved new best model')

Epoch 1:


Validation: 100%|██████████| 42/42 [00:04<00:00, 10.35it/s, loss=0.1456]


Validation loss: 0.14564063648382822
MAE per label:
	acousticness        0.2308
	danceability        0.1007
	energy              0.1258
	instrumentalness    0.2165
	liveness            0.0947
	speechiness         0.0532
	valence             0.1978
Saved new best model
Epoch 2:


Validation: 100%|██████████| 42/42 [00:04<00:00, 10.17it/s, loss=0.1456]


Validation loss: 0.14562499523162842
MAE per label:
	acousticness        0.2261
	danceability        0.1065
	energy              0.1249
	instrumentalness    0.2151
	liveness            0.0966
	speechiness         0.0516
	valence             0.1986
Saved new best model
Epoch 3:


...

Validation loss: 0.12937337905168533
MAE per label:
	acousticness        0.1942
	danceability        0.0932
	energy              0.1143
	instrumentalness    0.1949
	liveness            0.0898
	speechiness         0.0444
	valence             0.1749
Epoch 86:


Validation: 100%|██████████| 42/42 [00:06<00:00,  6.12it/s, loss=0.1293]


Validation loss: 0.12928109616041183
MAE per label:
	acousticness        0.1928
	danceability        0.0940
	energy              0.1155
	instrumentalness    0.1940
	liveness            0.0897
	speechiness         0.0446
	valence             0.1744
Saved new best model
Epoch 87:


Validation: 100%|██████████| 42/42 [00:06<00:00,  6.12it/s, loss=0.1292]


Validation loss: 0.12920704519464857
MAE per label:
	acousticness        0.1934
	danceability        0.0937
	energy              0.1146
	instrumentalness    0.1938
	liveness            0.0898
	speechiness         0.0445
	valence             0.1746


In [ ]:
model.load_state_dict(torch.load('best_model.pt'))

y, sr = librosa.load("16 Imagine Dragons - Believer.mp3", sr=16000)
log_mel = vggish_input.waveform_to_examples(y, sample_rate=16000).to(device).float()
log_mel = log_mel.squeeze(1)
log_mel = log_mel.unsqueeze(0)

model.eval()

with torch.inference_mode():
    with torch.autocast(device_type="cuda"):
        pred = model(log_mel)

pred = pred[0].cpu().numpy()
for i, label in enumerate(labels):
    print(f"{label:<20}{pred[i]:.4f}")

acousticness        0.6216
danceability        0.4705
energy              0.7773
instrumentalness    0.0137
liveness            0.1431
speechiness         0.0778
valence             0.6460
